# Hyperparameter Tuning using HyperDrive


In [1]:

# import dependencies
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

from azureml.core import ScriptRunConfig

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


In [3]:
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="capstone_project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-132094
Azure region: southcentralus
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-132094


## Dataset

In [4]:
# NOTE: update the key to match the dataset name
found = False
key = "diabetes-classification"
description_text = "Diabetes datasets for capstone project"

if key in ws.datasets.keys(): 
        found = True        
        dataset = ws.datasets[key] 

if not found:
        # Create Hyperdrive Dataset and register it into Workspace
        url = 'https://raw.githubusercontent.com/purunep/Capstone/main/project/data/diabetes.csv'
        dataset = Dataset.Tabular.from_delimited_files(url)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

##from train import clean_data
#[x_data, y_data] = clean_data(dataset)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "puru-compute-new"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import joblib

In [6]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
env = Environment('tutorial-env')
cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults'], conda_packages = ['scikit-learn==0.22.1'])

env.python.conda_dependencies = cd

# Register environment to re-use later
env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20200821.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "tutorial-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"

In [7]:
from azureml.core import ScriptRunConfig

args = ['--C', 0.5, '--max_iter', 10]

src = ScriptRunConfig(source_directory='.',
                      script='train.py', 
                      arguments=args,
                      compute_target=cpu_cluster,
                      environment=env)

In [8]:
run = exp.submit(config=src)
run

Experiment,Id,Type,Status,Details Page,Docs Page
capstone_project,capstone_project_1609112943_031f3fe8,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [ ]:
## Hyperdrive Configuration

In [11]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import joblib

# Specify parameter sampler
ps = RandomParameterSampling( {
        "--C": uniform(0.50,1.00),
       "--max_iter" : choice(10,20,30)
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)



#sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')
#envit = Environment.get(workspace=ws,name='myenv')

#config  = ScriptRunConfig(source_directory='.',
                         #script='train.py',                         
                         #compute_target=cpu_cluster,
                         #environment=env)



#hyperdrive_run_config = <your config here>
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                                   hyperparameter_sampling = ps,
                                   primary_metric_name = 'Accuracy',
                                   max_total_runs = 20,
                                   max_concurrent_runs = 4,
                                   primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                   policy = policy,
                                   run_config = src)

#estimator=estimator use to replace estimator in hyperdriveconfig

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()
# wait for completion
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_6d059a1d-10a1-434d-8a06-c3e670fff848
Web View: https://ml.azure.com/experiments/capstone_project/runs/HD_6d059a1d-10a1-434d-8a06-c3e670fff848?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-132094/workspaces/quick-starts-ws-132094

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-28T00:02:42.734670][API][INFO]Experiment created<END>\n""<START>[2020-12-28T00:02:43.262368][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2020-12-28T00:02:43.3918010Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2020-12-28T00:02:43.557025][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()


print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

In [ ]:
# register model for future deployment
# os.makedirs('outputs', exist_ok=True)

description = 'Diabetec'
tags = {'area': 'health', 'type': 'classification'}

hyperdrive_model = best_run.register_model(model_name='purumodel',
                                                      model_path='outputs/purumodel.pkl',
                                                      tags=tags,
                                                      description=description)

print(hyperdrive_model.name, hyperdrive_model.id, hyperdrive_model.version, sep='\t')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

In [ ]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
import joblib

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'purumodel.pkl')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()

In [ ]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "health",  "method" : "sklearn"}, 
                                               description='Predict diabetec with sklearn')

In [ ]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'purumodel')


myenv = Environment.get(workspace=ws, name="tutorial-env", version="1")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

service_name = 'puru-svc-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

In [ ]:
print(service.scoring_uri)

In [ ]:
model = Model(ws,name='purumodel')
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=4, enable_app_insights=True, auth_enabled=True)
inference_config = InferenceConfig(entry_script='score.py', environment=envit)

service = Model.deploy(ws, "puruservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.scoring_uri)
print(service.swagger_uri)

In [ ]:
service.get_logs()

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
primary, secondary = service.get_keys()
print(primary)

In [ ]:
import json


payload = json.dumps({
    'data': x_data[0:2].tolist(),
    'method': 'predict_proba'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(payload)

print(output)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# print logs
from azureml.core import Workspace
from azureml.core.webservice import Webservice

# Requires the config to be downloaded first to the current working directory
ws = Workspace.from_config()

# Set with the deployment name
name = "puruservice"

# load existing web service
service = Webservice(name=name, workspace=ws)
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

In [ ]:
service.delete()